#Step17 메모리 관리와 순환 참조
## 17.1 메모리 관리

파이썬에서 메모리를 관리는 두 가지 방식으로 진행된다.
1. 참조(reference)
2. 객체 회수 (Garbage Collection - GC)

- 참조 방식은 기본적으로 어떤 인스턴스가 참조(대입, 전달, 어떤 객체에 추가) 될 때 참조 카운트가 올라가는 방식이고, 참조가 풀릴 때 참조 카운트가 내려가는 방식이다.
- 하지만 이는 순환 참조 방식을 해결하지 못하는데, 이를 해결하기 위해 나온 것이 GC방식이다.

- GC는 참조 카운트와 달리 메모리가 부족해지는 시점에 파이썬 인터프리터에 의해 자동으로 호출된다.
- 명시적으로도 호줄 될 수 있는데, gc 모듈을 임포트해서 gc.collect()를 실행 하면 된다.
- GC는 순환참조를 올바르게 처리한다. 하지만 메모리 해제를 GC에 미루다 보면 프로그램의 전체 메모리 사용량이 (순환 참조가 없을 때와 비교해) 커지는 원인이 된다.
- 그래서 애초에 순환 참조를 만들지 않는 것이 중요하다.
- 현재 Variable 구조의 구현에는 순환 참조 되는 부분이 존재하는데, 이를 파이썬 모듈인 weakref로 해결해보자. 


## 17.4 weakref 모듈

- 파이썬에서는 weakref.ref 함수를 사용하여 약한 참조(weak reference)를 만들 수 있다.
- 약한 참조란 다른 객체를 참조하되 참조 카운트는 증가시키지 않는 기능이다.

In [1]:
import weakref
import numpy as np

a = np.array([1, 2, 3])
b = weakref.ref(a)

In [2]:
b # <weakref at 0x7fa533e06e90; to 'numpy.ndarray' at 0x7fa5335a9cb0>

<weakref at 0x7fa533e06e90; to 'numpy.ndarray' at 0x7fa5335a9cb0>

In [3]:
# 참조된 데이터에 접근하는 법
b() # array([1, 2, 3])

array([1, 2, 3])

In [5]:
a = None
b # <weakref at 0x7fa533e06e90; to 'numpy.ndarray' at 0x7fa5335a9cb0>
# 코랩의 파이썬 인터프리터에서는 b가 여전히 유효한 참조를 유지하는 것으로 보임.
# 원래는 <weakref at 0x7fa533e06e90; 'dead'>가 나와야한다.

<weakref at 0x7fa533e06e90; to 'numpy.ndarray' at 0x7fa5335a9cb0>

In [6]:
class Variable():
  def __init__(self, data):
    if data is not None:
      if not isinstance(data, np.ndarray):
        raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data)))

    self.data = data
    self.grad = None
    self.creator = None
    self.generation = 0 # 세대 수를 기록하는 변수 추가

  def set_creator(self, func):
    self.creator = func
    self.generation = func.generation + 1 # 세대를 기록(부모 세대 + 1)

  def backward(self):
    if self.grad is None:
      self.grad = np.ones_like(self.data)

    funcs = []
    seen_set = set()

    def add_func(f):
      if f not in seen_set:     # 같은 함수가 중복 추가 되는 것을 방지
                                # -> 두 elements가 같은 creator를 가지는 경우!
        funcs.append(f)
        seen_set.add(f)         
        funcs.sort(key=lambda x: x.generation)

    add_func(self.creator)
     
    while funcs :              
      f = funcs.pop()          
      gys = [output().grad for output in f.outputs] # output()(약한참조)의 grad 가져옴
      gxs = f.backward(*gys)                        # gxs를 구함
      if not isinstance(gxs, tuple):
        gxs = (gxs, )

      for x, gx in zip(f.inputs, gxs):            # 구한 역전파를 각 x.grad에 저장
        if x.grad is None:                        # 같은 변수일 경우에는 덮어쓰지 말고 더해주자!
          x.grad = gx
        else:
          x.grad = x.grad + gx

        if x.creator is not None:                 # 계속해서 역전파 수행
          add_func(x.creator)                     # x.creator 추가

def as_array(x):
  if np.isscalar(x):
    # x가 np.float64 같은 scalar 타입인지 확인(일반 float도 확인됨)
    return np.array(x)
  return x

class Function:
  def __call__(self, *inputs):
    xs = [x.data for x in inputs]
    ys = self.forward(*xs) # 언팩
    if not isinstance(ys, tuple): # tuple 형태가 아닐 경우 추가 지원
      ys = (ys,)
    outputs = [Variable(as_array(y)) for y in ys]
    self.generation = max([x.generation for x in inputs])

    for output in outputs:
      output.set_creator(self) 
    
    self.inputs = inputs
    self.outputs= [weakref.ref(output) for output in outputs] # self.outputs가 약한 참조로 가르키도록 변경
    return outputs if len(outputs) > 1 else outputs[0]

  def forward(self, x):
    raise NotImplementedError()

  def backward(self, gy):
    raise NotImplementedError()

def as_array(x):
  if np.isscalar(x):
    # x가 np.float64 같은 scalar 타입인지 확인(일반 float도 확인됨)
    return np.array(x)
  return x

class Add(Function):
  def forward(self, x0, x1):
    y = x0 + x1
    return y

  def backward(self, gy):
    '''
      backward 부분 추가
    '''
    return gy, gy

def add(x0, x1):
  return Add()(x0, x1)

class Square(Function):
  def forward(self, x):
    y = x ** 2
    return y

  def backward(self, gy):
    x = self.inputs[0].data # Function 함수에서 inputs (튜플)로 받기 때문에 input-> inputs[0]으로 바꿔줌
    gx = 2 * x * gy
    return gx

def square(x):
  return Square()(x)

In [7]:
for i in range(10):
  '''
    for문이 반복될 때 마다, x와 y는 이전의 메모리를 성공적으로 덮어씀.
    순환 참조가 발생하지만, 이미 약한 참조를 사용하고 있으므로 참조 카운트는 0이 됨
    이 시점에 계산 그래프에 사용된 메모리는 바로 삭제!

  ''' 
  x = Variable(np.random.randn(10000)) # 거대한 데이터
  y = square(square(square(x))) # 복잡한 계산 수행
